Good choice! MediaPipe is lightweight and efficient for real-time pose tracking.  

### **Next Steps:**  
1. **Set Up MediaPipe & OpenCV**  
   - Install necessary libraries:  
     ```bash
     pip install mediapipe opencv-python numpy
     ```
   - Capture live webcam feed  

2. **Detect Key Landmarks for Each Exercise**  
   - **Bicep Curls:** Track elbow and shoulder angles  
   - **Squats:** Monitor knee and hip angles  
   - **Push-ups:** Analyze elbow and shoulder alignment  

3. **Implement Rep Counting Logic**  
   - Identify full range of motion  
   - Count valid reps based on angle thresholds  

4. **Provide Real-time Feedback**  
   - Display **"Good Form"** or **"Adjust Position"** messages  

Would you like a base script to get started with MediaPipe Pose detection? 🚀

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Open the webcam
cap = cv2.VideoCapture(0)

# Select exercise to track
exercise = "bicep curl"  # Change this to "squat" or "pushup" as needed
count = 0  # Counter for repetitions
position = None  # To track movement direction

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip image horizontally for better visualization
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for MediaPipe
    results = pose.process(image)  # Process image with pose detection
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert back to BGR for OpenCV display
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark  # Get pose landmarks
        
        # Detect and analyze key points for different exercises
        if exercise == "bicep curl":
            # Get coordinates of key joints
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate elbow angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Rep counting logic
            if angle > 140:
                position = "down"
            if angle < 40 and position == "down":
                position = "up"
                count += 1  # Increment rep count
            
            # Provide feedback based on form
            feedback = "Good Form" if 30 <= angle <= 150 else "Fix Arm Position"
        
        elif exercise == "squat":
            # Get coordinates of key joints
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate knee angle
            angle = calculate_angle(hip, knee, ankle)
            
            # Rep counting logic
            if angle > 160:
                position = "up"
            if angle < 90 and position == "up":
                position = "down"
                count += 1  # Increment rep count
            
            # Provide feedback based on form
            feedback = "Good Form" if 80 <= angle <= 170 else "Fix Knee Position"
        
        elif exercise == "pushup":
            # Get coordinates of key joints
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate elbow angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Rep counting logic
            if angle > 160:
                position = "up"
            if angle < 90 and position == "up":
                position = "down"
                count += 1  # Increment rep count
            
            # Provide feedback based on form
            feedback = "Good Form" if 80 <= angle <= 170 else "Fix Elbow Position"
        
        # Draw pose landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Display repetition count and feedback on the screen
        cv2.putText(image, f"Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(image, feedback, (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Show the image with pose tracking
    cv2.imshow("Fitoproto - AI Rep Counter", image)
    
    # Exit when 'q' key is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


## Live Exercise Feedback and angle detection


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Open the webcam
cap = cv2.VideoCapture(0)

def draw_concentration_bar(image, count):
    bar_height = int(400 * (count % 10) / 10)  # Simple visualization based on reps
    cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
    cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

def select_exercise():
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl()
    elif choice == "2":
        squat()
    elif choice == "3":
        pushup()
    else:
        print("Invalid choice. Please restart and enter a valid number.")

def bicep_curl():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            angle = calculate_angle(shoulder, elbow, wrist)
            
            if angle > 140:
                position = "down"
            if angle < 40 and position == "down":
                position = "up"
                count += 1
            
            feedback = "Good form!" if 30 < angle < 140 else "Adjust your arm position."
            
            cv2.putText(image, f"Bicep Curl Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        draw_concentration_bar(image, count)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow("Bicep Curl", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

def squat():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            angle = calculate_angle(hip, knee, ankle)
            
            if angle > 160:
                position = "up"
            if angle < 90 and position == "up":
                position = "down"
                count += 1
            
            feedback = "Good depth!" if 80 < angle < 160 else "Go lower!"
            
            cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        draw_concentration_bar(image, count)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow("Squat", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

def pushup():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            angle = calculate_angle(shoulder, elbow, wrist)
            
            if angle > 160:
                position = "up"
            if angle < 90 and position == "up":
                position = "down"
                count += 1
            
            feedback = "Good form!" if 80 < angle < 160 else "Keep your core tight!"
            
            cv2.putText(image, f"Pushup Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        draw_concentration_bar(image, count)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow("Pushup", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

select_exercise()


Select an exercise:
1. Bicep Curl
2. Squat
3. Pushup


: 

# Input video output


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Function to draw a simple concentration bar
def draw_concentration_bar(image, count):
    bar_height = int(400 * (count % 10) / 10)  # Simple visualization based on reps
    cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
    cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

# Function to select input source (Webcam or Video file)
def select_input_source():
    print("Select input source:")
    print("1. Live Webcam")
    print("2. Video File")
    choice = input("Enter 1 or 2: ")
    if choice == "1":
        return cv2.VideoCapture(0)  # Open webcam
    elif choice == "2":
        video_path = input("Enter video file path: ")
        return cv2.VideoCapture(video_path)  # Open video file
    else:
        print("Invalid choice. Defaulting to webcam.")
        return cv2.VideoCapture(0)

# Function to select exercise type
def select_exercise(cap):
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl(cap)
    elif choice == "2":
        squat(cap)
    elif choice == "3":
        pushup(cap)
    else:
        print("Invalid choice. Please restart and enter a valid number.")

# Function to process video frames and track reps
def process_video(cap, exercise_type):
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)  # Mirror the frame
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Define joints for different exercises
            if exercise_type == "bicep_curl":
                joint1, joint2, joint3 = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST]
            elif exercise_type == "squat":
                joint1, joint2, joint3 = [mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]
            elif exercise_type == "pushup":
                joint1, joint2, joint3 = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST]
            
            # Get joint positions
            p1 = [landmarks[joint1.value].x, landmarks[joint1.value].y]
            p2 = [landmarks[joint2.value].x, landmarks[joint2.value].y]
            p3 = [landmarks[joint3.value].x, landmarks[joint3.value].y]
            
            # Calculate joint angle
            angle = calculate_angle(p1, p2, p3)
            
            # Rep counting logic based on exercise type
            if exercise_type == "bicep_curl":
                if angle > 140:
                    position = "down"
                if angle < 40 and position == "down":
                    position = "up"
                    count += 1
                feedback = "Good form!" if 30 < angle < 140 else "Adjust your arm position."
            
            elif exercise_type == "squat":
                if angle > 160:
                    position = "up"
                if angle < 90 and position == "up":
                    position = "down"
                    count += 1
                feedback = "Good depth!" if 80 < angle < 160 else "Go lower!"
            
            elif exercise_type == "pushup":
                if angle > 160:
                    position = "up"
                if angle < 90 and position == "up":
                    position = "down"
                    count += 1
                feedback = "Good form!" if 80 < angle < 160 else "Keep your core tight!"
            
            # Display rep count and feedback
            cv2.putText(image, f"Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # Draw concentration bar
        draw_concentration_bar(image, count)
        
        # Draw landmarks on image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Show the processed frame
        cv2.imshow(exercise_type.capitalize(), image)
        
        # Press 'q' to quit
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Functions to call process_video with specific exercises
def bicep_curl(cap):
    process_video(cap, "bicep_curl")

def squat(cap):
    process_video(cap, "squat")

def pushup(cap):
    process_video(cap, "pushup")

# Start program
cap = select_input_source()
select_exercise(cap)


Select input source:
1. Live Webcam
2. Video File
Invalid choice. Defaulting to webcam.
Select an exercise:
1. Bicep Curl
2. Squat
3. Pushup
Invalid choice. Please restart and enter a valid number.


## Live Updated

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime
import time


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    
    # Check if the file exists, if not, add headers
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists, no need to write headers
    
    # Append new data
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])
    
    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")

# to close the program much better 
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()  # Immediate program termination


# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# open the webcam
cap = cv2.VideoCapture(0)

# def draw_concentration_bar(image, count):
#     bar_height = int(368 * (count % 12) / 10)  # Simple visualization based on reps
#     cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
#     cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

def select_exercise():
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl()
    elif choice == "2":
        squat()
    elif choice == "3":
        pushup()
    else:
        print("Invalid choice. Please restart and enter a valid number.")

# to fix the issue in to much quick response
#  Updated Code with Delayed Feedback Switch
# Initialize smoothing variables

smoothed_fill_ratio = 0  # Store the smoothed value
alpha = 0.1  # Smoothing factor (Higher = Faster updates, Lower = Smoother)

def draw_concentration_bar(image, angle):
    """
    Draws a smoothed concentration bar based on squat depth.
    Uses Exponential Moving Average (EMA) for smooth transitions.
    """

    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill ratio (160° → 0%, 80° → 100%)
    target_fill_ratio = max(0, min(1, (160 - angle) / 80))  

    # Apply smoothing (Exponential Moving Average)
    # Adjust smoothing factor to make bar update faster
    alpha = 0.25  # Increased from 0.1 to 0.25 for a quicker response
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)  

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

# Global variable for bar smoothing
smoothed_fill_ratio = 0  

def draw_concentration_bar_biceps(image, angle):
    """Draws a smoothed concentration bar based on curl depth using EMA."""
    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  
    bar_y = 100  
    bar_height = 200  
    bar_width = 30  

    # Normalize angle to fill ratio (150° → 0%, 30° → 100%)
    target_fill_ratio = max(0, min(1, (150 - angle) / 120))  

    # Apply smoothing (Exponential Moving Average)
    alpha = 0.25  # Smoothing factor
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not curled enough)
    elif 70 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent curl)
    else:
        color = (0, 255, 0)  # Green (Perfect curl)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Curl", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def bicep_curl():
    """Tracks bicep curls and counts reps."""
    count = 0
    position = "down"

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Get keypoints for right arm
                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    
                    # add key pint for left also 
                    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    #right angle 
                    r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                    #left angle
                    l_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)

                    # DEBUG: Print values
                    print(f"Angle: {r_angle:.2f}, Position: {position}")

                    # Check bicep curl position logic 
                    #for both left and right hand
                    if r_angle > 140 and l_angle > 140:
                        position = "down"

                    if r_angle < 50 and l_angle < 50 and position == "down":
                        position = "up"
                        count += 1
                        print(f"✅ Curl Counted! Total: {count}")  

                    # Provide feedback
                    if r_angle > 140 and l_angle > 140:
                        feedback = "Extend your arm!"
                    elif r_angle < 50 :
                        feedback = "Full curl!"
                    else:
                        feedback = "Good form!"

                    # Display feedback
                    cv2.putText(image, f"Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    # Draw the concentration bar
                    draw_concentration_bar_biceps(image,  (r_angle + l_angle) / 2)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Bicep Curl", image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Bicep curl",count)
                cleanup()
                break

    # cap.release()
    # cv2.destroyAllWindows()


def draw_concentration_bar_squat(image, angle):
    """
    Draws a concentration bar on the screen based on the squat depth (angle).
    - More filled when squatting lower.
    - Color changes based on depth.
    """

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position of the bar
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill the bar
    fill_ratio = max(0, min(1, (160 - angle) / 80))  # 160° (empty) → 80° (full)
    fill_height = int(fill_ratio * bar_height)  # Compute filled height

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw text labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def squat():
    count = 0
    position = "up"
    shoulder_initial_y = None
    last_feedback = "Start your exercise"
    last_feedback_time = time.time()
    feedback_delay = 1.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

                angle = calculate_angle(hip, knee, ankle)

                if position == "up" and shoulder_initial_y is None:
                    shoulder_initial_y = shoulder[1]

                print(f"Angle: {angle:.2f}, Position: {position}")

                if angle > 160:
                    position = "up"
                    shoulder_initial_y = shoulder[1]

                if angle < 90 and position == "up" and shoulder[1] > shoulder_initial_y + 0.02:
                    position = "down"
                    count += 1
                    print(f"✅ Squat Counted! Total: {count}")

                new_feedback = last_feedback
                if angle > 160:
                    new_feedback = "Stand tall!"
                elif angle < 90:
                    new_feedback = "Squat low!"
                else:
                    new_feedback = "Good depth!"

                if time.time() - last_feedback_time > feedback_delay:
                    last_feedback = new_feedback
                    last_feedback_time = time.time()

                # Draw smoothed concentration bar
                draw_concentration_bar_squat(image, angle)

                cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, last_feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Squat", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Squats",count)
            cleanup()
            break

    # cap.release()
    # cv2.destroyAllWindows()


## solved by adding the shouder points 
## wrong counting


def pushup():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                angle = calculate_angle(shoulder, elbow, wrist)

                if angle > 160:
                    position = "up"
                if angle < 90 and position == "up":
                    position = "down"
                    count += 1
                
                feedback = "Good form!" if 80 < angle < 160 else "Keep your core tight!"
                
                cv2.putText(image, f"Pushup Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        draw_concentration_bar(image, count)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Pushup", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Pushup", count)
            cleanup()


## solved by adding the shouder points 

# def squat():
#     count = 0
#     position = "up"
#     shoulder_initial_y = None  # Stores the reference height of the shoulder

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         frame = cv2.flip(frame, 1)
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         results = pose.process(image)
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         try:
#             if results.pose_landmarks:
#                 landmarks = results.pose_landmarks.landmark

#                 # Get keypoints
#                 hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
#                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
#                 knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
#                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
#                 ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
#                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
#                 shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
#                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

#                 angle = calculate_angle(hip, knee, ankle)

#                 # Initialize shoulder reference height when standing straight
#                 if position == "up" and shoulder_initial_y is None:
#                     shoulder_initial_y = shoulder[1]

#                 # DEBUG: Print values
#                 print(f"Angle: {angle:.2f}, Shoulder Y: {shoulder[1]:.2f}, Initial Y: {shoulder_initial_y}, Position: {position}")

#                 # Check squat position logic
#                 if angle > 160:  # Standing position
#                     position = "up"
#                     shoulder_initial_y = shoulder[1]  # Reset shoulder reference when standing

#                 if angle < 90 and position == "up" and shoulder[1] > shoulder_initial_y + 0.02:  
#                     # Ensure the shoulder moves down before counting the squat
#                     position = "down"
#                     count += 1
#                     print(f"✅ Squat Counted! Total: {count}")  # Debugging line

#                 # Provide feedback
#                 if angle > 160:
#                     feedback = "Stand tall!"
#                 elif angle < 90:
#                     feedback = "Squat low!"
#                 else:
#                     feedback = "Good depth!"

#                 # Display feedback
#                 cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#                 cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

#         except Exception as e:
#             print(f"Error: {e}")

#         # Draw landmarks
#         if results.pose_landmarks:
#             mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#         cv2.imshow("Squat", image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

select_exercise()


Select an exercise:
1. Bicep Curl
2. Squat
3. Pushup
Angle: 167.29, Position: down
Angle: 165.54, Position: down
Angle: 165.67, Position: down
Angle: 164.13, Position: down
Angle: 163.37, Position: down
Angle: 163.96, Position: down
Angle: 163.28, Position: down
Angle: 163.24, Position: down
Angle: 163.18, Position: down
Angle: 161.93, Position: down
Angle: 163.30, Position: down
Angle: 164.58, Position: down
Angle: 164.05, Position: down
Angle: 164.73, Position: down
Angle: 165.30, Position: down
Angle: 165.26, Position: down
Angle: 164.83, Position: down
Angle: 165.74, Position: down
Angle: 166.84, Position: down
Angle: 168.19, Position: down
Angle: 168.54, Position: down
Angle: 168.80, Position: down
Angle: 167.99, Position: down
Angle: 167.00, Position: down
Angle: 166.53, Position: down
Angle: 166.83, Position: down
Angle: 166.92, Position: down
Angle: 166.21, Position: down
Angle: 165.32, Position: down
Angle: 165.40, Position: down
Angle: 165.77, Position: down
Angle: 166.53, Po

: 

# Finished


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime
import time


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    
    # Check if the file exists, if not, add headers
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists, no need to write headers
    
    # Append new data
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])
    
    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")

# to close the program much better 
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()  # Immediate program termination


# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# open the webcam
cap = cv2.VideoCapture(0)

# def draw_concentration_bar(image, count):
#     bar_height = int(368 * (count % 12) / 10)  # Simple visualization based on reps
#     cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
#     cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

def select_exercise():
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl()
    elif choice == "2":
        squat()
    elif choice == "3":
        pushup()
    else:
        print("Invalid choice. Please restart and enter a valid number.")

# to fix the issue in to much quick response
#  Updated Code with Delayed Feedback Switch
# Initialize smoothing variables

smoothed_fill_ratio = 0  # Store the smoothed value
alpha = 0.1  # Smoothing factor (Higher = Faster updates, Lower = Smoother)

def draw_concentration_bar(image, angle):
    """
    Draws a smoothed concentration bar based on squat depth.
    Uses Exponential Moving Average (EMA) for smooth transitions.
    """

    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill ratio (160° → 0%, 80° → 100%)
    target_fill_ratio = max(0, min(1, (160 - angle) / 80))  

    # Apply smoothing (Exponential Moving Average)
    # Adjust smoothing factor to make bar update faster
    alpha = 0.25  # Increased from 0.1 to 0.25 for a quicker response
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)  

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

# Global variable for bar smoothing
smoothed_fill_ratio = 0  

def draw_concentration_bar_biceps(image, angle):
    """Draws a smoothed concentration bar based on curl depth using EMA."""
    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  
    bar_y = 100  
    bar_height = 200  
    bar_width = 30  

    # Normalize angle to fill ratio (150° → 0%, 30° → 100%)
    target_fill_ratio = max(0, min(1, (150 - angle) / 120))  

    # Apply smoothing (Exponential Moving Average)
    alpha = 0.25  # Smoothing factor
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not curled enough)
    elif 70 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent curl)
    else:
        color = (0, 255, 0)  # Green (Perfect curl)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Curl", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def bicep_curl():
    """Tracks bicep curls and counts reps."""
    count = 0
    position = "down"

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Get keypoints for right arm
                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    
                    # add key pint for left also 
                    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    #right angle 
                    r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                    #left angle
                    l_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)

                    # DEBUG: Print values
                    print(f"Angle: {r_angle:.2f}, Position: {position}")

                    # Check bicep curl position logic 
                    #for both left and right hand
                    if r_angle > 140 and l_angle > 140:
                        position = "down"

                    if r_angle < 50 and l_angle < 50 and position == "down":
                        position = "up"
                        count += 1
                        print(f"✅ Curl Counted! Total: {count}")  

                    # Provide feedback
                    if r_angle > 140 and l_angle > 140:
                        feedback = "Extend your arm!"
                    elif r_angle < 50 :
                        feedback = "Full curl!"
                    else:
                        feedback = "Good form!"

                    # Display feedback
                    cv2.putText(image, f"Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    # Draw the concentration bar
                    draw_concentration_bar_biceps(image,  (r_angle + l_angle) / 2)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Bicep Curl", image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Bicep curl",count)
                cleanup()
                break

    # cap.release()
    # cv2.destroyAllWindows()


def draw_concentration_bar_squat(image, angle):
    """
    Draws a concentration bar on the screen based on the squat depth (angle).
    - More filled when squatting lower.
    - Color changes based on depth.
    """

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position of the bar
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill the bar
    fill_ratio = max(0, min(1, (160 - angle) / 80))  # 160° (empty) → 80° (full)
    fill_height = int(fill_ratio * bar_height)  # Compute filled height

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw text labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def squat():
    count = 0
    position = "up"
    shoulder_initial_y = None
    last_feedback = "Start your exercise"
    last_feedback_time = time.time()
    feedback_delay = 1.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

                angle = calculate_angle(hip, knee, ankle)

                if position == "up" and shoulder_initial_y is None:
                    shoulder_initial_y = shoulder[1]

                print(f"Angle: {angle:.2f}, Position: {position}")

                if angle > 160:
                    position = "up"
                    shoulder_initial_y = shoulder[1]

                if angle < 90 and position == "up" and shoulder[1] > shoulder_initial_y + 0.02:
                    position = "down"
                    count += 1
                    print(f"✅ Squat Counted! Total: {count}")

                new_feedback = last_feedback
                if angle > 160:
                    new_feedback = "Stand tall!"
                elif angle < 90:
                    new_feedback = "Squat low!"
                else:
                    new_feedback = "Good depth!"

                if time.time() - last_feedback_time > feedback_delay:
                    last_feedback = new_feedback
                    last_feedback_time = time.time()

                # Draw smoothed concentration bar
                draw_concentration_bar_squat(image, angle)

                cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, last_feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Squat", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Squats",count)
            cleanup()
            break

    # cap.release()
    # cv2.destroyAllWindows()


## solved by adding the shouder points 
## wrong counting


def pushup():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                angle = calculate_angle(shoulder, elbow, wrist)

                if angle > 160:
                    position = "up"
                if angle < 90 and position == "up":
                    position = "down"
                    count += 1
                
                feedback = "Good form!" if 80 < angle < 160 else "Keep your core tight!"
                
                cv2.putText(image, f"Pushup Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        draw_concentration_bar(image, count)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Pushup", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Pushup", count)
            cleanup()


select_exercise()


Select an exercise:
1. Bicep Curl
2. Squat
3. Pushup
Angle: 177.36, Position: down
Angle: 176.51, Position: down
Angle: 178.22, Position: down
Angle: 178.77, Position: down
Angle: 178.30, Position: down
Angle: 177.68, Position: down
Angle: 178.79, Position: down
Angle: 179.69, Position: down
Angle: 178.62, Position: down
Angle: 177.55, Position: down
Angle: 176.70, Position: down
Angle: 177.09, Position: down
Angle: 175.09, Position: down
Angle: 173.11, Position: down
Angle: 173.34, Position: down
Angle: 179.54, Position: down
Angle: 178.88, Position: down
Angle: 178.42, Position: down
Angle: 178.52, Position: down
Angle: 173.98, Position: down
Angle: 176.87, Position: down
Angle: 175.07, Position: down
Angle: 177.89, Position: down
Angle: 173.92, Position: down
Angle: 170.45, Position: down
Angle: 166.59, Position: down
Angle: 163.45, Position: down
Angle: 164.46, Position: down
Angle: 172.89, Position: down
Angle: 168.77, Position: down
Angle: 169.40, Position: down
Angle: 171.73, Po

: 

# Trying

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import datetime
import time


# Function to log exercise reps in CSV
def log_reps(exercise_name, count):
    filename = "exercise_log.csv"
    
    # Check if the file exists, if not, add headers
    try:
        with open(filename, 'x', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Time", "Exercise", "Reps"])
    except FileExistsError:
        pass  # File already exists, no need to write headers
    
    # Append new data
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        writer.writerow([timestamp, exercise_name, count])
    
    print(f"✅ Saved {count} reps of {exercise_name} to exercise_log.csv")

# to close the program much better 
def cleanup():
    cap.release()
    cv2.destroyAllWindows()
    exit()  # Immediate program termination


# Function to calculate the angle between three points (for joint angle calculations)
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint (joint)
    c = np.array(c)  # End point
    
    # Calculate the angle using arctan2 function
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Adjust angle to stay within 0-180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Get video frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = 20  # Adjust FPS as needed

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Use 'MP4V' for .mp4
out = cv2.VideoWriter("output.mp4", fourcc, fps, (frame_width, frame_height))
# open the webcam
cap = cv2.VideoCapture(0)

# def draw_concentration_bar(image, count):
#     bar_height = int(368 * (count % 12) / 10)  # Simple visualization based on reps
#     cv2.rectangle(image, (50, 450 - bar_height), (100, 450), (0, 255, 0), -1)
#     cv2.rectangle(image, (50, 50), (100, 450), (255, 255, 255), 2)

def select_exercise():
    print("Select an exercise:")
    print("1. Bicep Curl")
    print("2. Squat")
    print("3. Pushup")
    choice = input("Enter 1, 2, or 3: ")
    if choice == "1":
        bicep_curl()
    elif choice == "2":
        squat()
    elif choice == "3":
        pushup()
    else:
        print("Invalid choice. Please restart and enter a valid number.")

# to fix the issue in to much quick response
#  Updated Code with Delayed Feedback Switch
# Initialize smoothing variables

smoothed_fill_ratio = 0  # Store the smoothed value
alpha = 0.1  # Smoothing factor (Higher = Faster updates, Lower = Smoother)

def draw_concentration_bar(image, angle):
    """
    Draws a smoothed concentration bar based on squat depth.
    Uses Exponential Moving Average (EMA) for smooth transitions.
    """

    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill ratio (160° → 0%, 80° → 100%)
    target_fill_ratio = max(0, min(1, (160 - angle) / 80))  

    # Apply smoothing (Exponential Moving Average)
    # Adjust smoothing factor to make bar update faster
    alpha = 0.25  # Increased from 0.1 to 0.25 for a quicker response
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)  

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

# Global variable for bar smoothing
smoothed_fill_ratio = 0  

def draw_concentration_bar_biceps(image, angle):
    """Draws a smoothed concentration bar based on curl depth using EMA."""
    global smoothed_fill_ratio

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  
    bar_y = 100  
    bar_height = 200  
    bar_width = 30  

    # Normalize angle to fill ratio (150° → 0%, 30° → 100%)
    target_fill_ratio = max(0, min(1, (150 - angle) / 120))  

    # Apply smoothing (Exponential Moving Average)
    alpha = 0.25  # Smoothing factor
    smoothed_fill_ratio = alpha * target_fill_ratio + (1 - alpha) * smoothed_fill_ratio

    # Compute filled height
    fill_height = int(smoothed_fill_ratio * bar_height)

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not curled enough)
    elif 70 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent curl)
    else:
        color = (0, 255, 0)  # Green (Perfect curl)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion (smoothed)
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw labels
    cv2.putText(image, "Curl", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def bicep_curl():
    """Tracks bicep curls and counts reps."""
    count = 0
    position = "down"

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Get keypoints for right arm
                    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    
                    # add key pint for left also 
                    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    #right angle 
                    r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                    #left angle
                    l_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)

                    # DEBUG: Print values
                    print(f"Angle: {r_angle:.2f}, Position: {position}")

                    # Check bicep curl position logic 
                    #for both left and right hand
                    if r_angle > 140 and l_angle > 140:
                        position = "down"

                    if r_angle < 50 and l_angle < 50 and position == "down":
                        position = "up"
                        count += 1
                        print(f"✅ Curl Counted! Total: {count}")  

                    # Provide feedback
                    if r_angle > 140 and l_angle > 140:
                        feedback = "Extend your arm!"
                    elif r_angle < 50 :
                        feedback = "Full curl!"
                    else:
                        feedback = "Good form!"

                    # Display feedback
                    cv2.putText(image, f"Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    # Draw the concentration bar
                    draw_concentration_bar_biceps(image,  (r_angle + l_angle) / 2)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Bicep Curl", image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                log_reps("Bicep curl",count)
                cleanup()
                break

    # cap.release()
    # cv2.destroyAllWindows()


def draw_concentration_bar_squat(image, angle):
    """
    Draws a concentration bar on the screen based on the squat depth (angle).
    - More filled when squatting lower.
    - Color changes based on depth.
    """

    h, w, _ = image.shape  # Get image dimensions

    # Define bar properties
    bar_x = 50  # X position of the bar
    bar_y = 100  # Y position (top)
    bar_height = 200  # Max height
    bar_width = 30  # Width

    # Normalize angle to fill the bar
    fill_ratio = max(0, min(1, (160 - angle) / 80))  # 160° (empty) → 80° (full)
    fill_height = int(fill_ratio * bar_height)  # Compute filled height

    # Choose color based on depth
    if angle > 140:
        color = (0, 0, 255)  # Red (Not deep enough)
    elif 90 <= angle <= 140:
        color = (0, 255, 255)  # Yellow (Decent)
    else:
        color = (0, 255, 0)  # Green (Perfect squat)

    # Draw empty bar (background)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)

    # Draw filled portion
    cv2.rectangle(image, (bar_x, bar_y + (bar_height - fill_height)), (bar_x + bar_width, bar_y + bar_height), color, -1)

    # Draw text labels
    cv2.putText(image, "Depth", (bar_x - 10, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(image, "Low", (bar_x + 40, bar_y + bar_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(image, "High", (bar_x + 40, bar_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def squat():
    count = 0
    position = "up"
    shoulder_initial_y = None
    last_feedback = "Start your exercise"
    last_feedback_time = time.time()
    feedback_delay = 1.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

                angle = calculate_angle(hip, knee, ankle)

                if position == "up" and shoulder_initial_y is None:
                    shoulder_initial_y = shoulder[1]

                print(f"Angle: {angle:.2f}, Position: {position}")

                if angle > 160:
                    position = "up"
                    shoulder_initial_y = shoulder[1]

                if angle < 90 and position == "up" and shoulder[1] > shoulder_initial_y + 0.02:
                    position = "down"
                    count += 1
                    print(f"✅ Squat Counted! Total: {count}")

                new_feedback = last_feedback
                if angle > 160:
                    new_feedback = "Stand tall!"
                elif angle < 90:
                    new_feedback = "Squat low!"
                else:
                    new_feedback = "Good depth!"

                if time.time() - last_feedback_time > feedback_delay:
                    last_feedback = new_feedback
                    last_feedback_time = time.time()

                # Draw smoothed concentration bar
                draw_concentration_bar_squat(image, angle)

                cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, last_feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Squat", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Squats",count)
            cleanup()
            break

    # cap.release()
    # cv2.destroyAllWindows()


## solved by adding the shouder points 
## wrong counting


def pushup():
    count = 0
    position = None
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                angle = calculate_angle(shoulder, elbow, wrist)

                if angle > 160:
                    position = "up"
                if angle < 90 and position == "up":
                    position = "down"
                    count += 1
                
                feedback = "Good form!" if 80 < angle < 160 else "Keep your core tight!"
                
                cv2.putText(image, f"Pushup Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        draw_concentration_bar(image, count)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Pushup", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            log_reps("Pushup", count)
            cleanup()


select_exercise()


KeyboardInterrupt: 

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize video capture
cap = cv2.VideoCapture(0)

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # Hip
    b = np.array(b)  # Knee
    c = np.array(c)  # Ankle

    # Vector calculations
    ba = a - b
    bc = c - b

    # Calculate angle
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Squat detection function
def squat():
    count = 0
    position = "up"
    shoulder_initial_y = None  # Stores the reference height of the shoulder

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # Get keypoints
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

                angle = calculate_angle(hip, knee, ankle)

                # Initialize shoulder reference height when standing straight
                if position == "up" and shoulder_initial_y is None:
                    shoulder_initial_y = shoulder[1]

                # DEBUG: Print values
                print(f"Angle: {angle:.2f}, Shoulder Y: {shoulder[1]:.2f}, Initial Y: {shoulder_initial_y}, Position: {position}")

                # Check squat position logic
                if angle > 160:  # Standing position
                    position = "up"
                    shoulder_initial_y = shoulder[1]  # Reset shoulder reference when standing

                if angle < 90 and position == "up" and shoulder[1] > shoulder_initial_y + 0.02:  
                    # Ensure the shoulder moves down before counting the squat
                    position = "down"
                    count += 1
                    print(f"✅ Squat Counted! Total: {count}")  # Debugging line

                # Provide feedback
                if angle > 160:
                    feedback = "Stand tall!"
                elif angle < 90:
                    feedback = "Squat low!"
                else:
                    feedback = "Good depth!"

                # Display feedback
                cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Squat", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the squat function
squat()


Angle: 177.67, Shoulder Y: 0.67, Initial Y: 0.6740996837615967, Position: up
Angle: 177.64, Shoulder Y: 0.67, Initial Y: 0.6740996837615967, Position: up
Angle: 177.63, Shoulder Y: 0.68, Initial Y: 0.67423415184021, Position: up
Angle: 177.56, Shoulder Y: 0.68, Initial Y: 0.6789358854293823, Position: up
Angle: 177.16, Shoulder Y: 0.68, Initial Y: 0.6808309555053711, Position: up
Angle: 176.80, Shoulder Y: 0.68, Initial Y: 0.6822826862335205, Position: up
Angle: 176.79, Shoulder Y: 0.68, Initial Y: 0.6843658089637756, Position: up
Angle: 176.48, Shoulder Y: 0.68, Initial Y: 0.6845875978469849, Position: up
Angle: 176.51, Shoulder Y: 0.68, Initial Y: 0.6763670444488525, Position: up
Angle: 176.43, Shoulder Y: 0.68, Initial Y: 0.676646888256073, Position: up
Angle: 176.54, Shoulder Y: 0.68, Initial Y: 0.6769821643829346, Position: up
Angle: 176.26, Shoulder Y: 0.67, Initial Y: 0.6772587299346924, Position: up
Angle: 176.16, Shoulder Y: 0.67, Initial Y: 0.6740161776542664, Position: up
An

import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize video capture
cap = cv2.VideoCapture(0)

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # Hip
    b = np.array(b)  # Knee
    c = np.array(c)  # Ankle

    # Vector calculations
    ba = a - b
    bc = c - b

    # Calculate angle
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Squat detection function
def squat():
    count = 0
    position = "up"
    shoulder_initial_y = None  # Stores the reference height of the shoulder

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # Get keypoints
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

                angle = calculate_angle(hip, knee, ankle)

                # Initialize shoulder reference height when standing straight
                if position == "up" and shoulder_initial_y is None:
                    shoulder_initial_y = shoulder[1]

                # DEBUG: Print values
                print(f"Angle: {angle:.2f}, Shoulder Y: {shoulder[1]:.2f}, Initial Y: {shoulder_initial_y}, Position: {position}")

                # Check squat position logic
                if angle > 160:  # Standing position
                    position = "up"
                    shoulder_initial_y = shoulder[1]  # Reset shoulder reference when standing

                if angle < 90 and position == "up" and shoulder[1] > shoulder_initial_y + 0.02:  
                    # Ensure the shoulder moves down before counting the squat
                    position = "down"
                    count += 1
                    print(f"✅ Squat Counted! Total: {count}")  # Debugging line

                # Provide feedback
                if angle > 160:
                    feedback = "Stand tall!"
                elif angle < 90:
                    feedback = "Squat low!"
                else:
                    feedback = "Good depth!"

                # Display feedback
                cv2.putText(image, f"Squat Reps: {count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(image, feedback, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error: {e}")

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Squat", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the squat function
squat()
